In [ ]:
import requests as req
import pandas as pd
import sqlite3

In [ ]:
endpoint_vacancies = 'https://api.hh.ru/vacancies'
api_headers = {'User-Agent': 'Mozilla/5.0',
               'Authorization': 'Bearer YOUR_API_KEY'}
DB_NAME = 'api.db'
TABLE_NAME = 'vacancies'

In [ ]:

api_params = {'text':'python middle',
              'per_page':25,
              'page':0}

In [ ]:
vacancyDetailInfo =[]

In [ ]:
def parsJsonAppendData (data):
    for result in data['items']:
        vacancyRow = []
        listOfRequirements = []
        vacancyName = result['name']
        companyName = result['employer']['name']
        vacancyid = result['id']
        vacancyDetail = req.get(endpoint_vacancies+'/'+vacancyid).json()
        vacancyDescription = vacancyDetail['description']
        for skill in vacancyDetail['key_skills']:
            listOfRequirements.append(skill['name'])
        vacancyRow.append(companyName)
        vacancyRow.append(vacancyName)
        vacancyRow.append(vacancyDescription)
        vacancyRow.append(listOfRequirements)
        vacancyDetailInfo.append(vacancyRow)

In [ ]:
while (True):
    results = req.get(endpoint_vacancies, params = api_params).json()
    if (len(results['items']) == 0):
        break
    parsJsonAppendData(results)
    print('Записей в итоговом результате',len(vacancyDetailInfo))
    api_params['page'] += 1
    

In [ ]:
columns = ['company_name', 'position', 'job_description', 'key_skills']
df = pd.DataFrame(data=vacancyDetailInfo,columns=columns,index=None)

In [20]:
df

,company_name,position,job_description,key_skills
0,Интернет вещей,Junior/Middle Python разработчик,"<p>Мы ищем опытного Python разработчика, котор...","[Python, PostgreSQL, API, REST API, Django, Dj..."
1,TerraLink,Python разработчик,<p><em>Компания</em><em> ТерраЛинк </em><em>– ...,[]
2,ГРЕЧКА,Backend - разработчик на Python,<p>Мы - Компания <strong>Гречка.Digital</stron...,"[Python, Django Framework, Git, PostgreSQL, My..."
3,НИИАС,Junior/Middle Python-developer,<p><strong>АО «НИИАС» (дочернее общество ОАО «...,[]
4,Notamedia,Manual QA инженер (middle),"<p>Notamedia — аккредитованный ИТ-холдинг, оди...","[Тестирование, Postman, Swagger, Charles, Atla..."
...,...,...,...,...
899,Skillbox,Спикер на курс «Автотесты на Python. Продвинут...,<p><strong>Skillbox</strong> – крупнейшая обра...,"[Обучение, Подготовка презентаций, Навыки през..."
900,HeadHunter,Middle/Senior Продуктовый аналитик в направлен...,<p><strong>Продуктовая команда HeadHunter в по...,"[SQL, Tableau, Python, Hadoop, Analytics / DS,..."
901,Контур,Middle DevOps-инженер в Контур.Толк,<p>Контур — экосистема для бизнеса. Мы создаем...,"[Linux, Ansible, Kubernetes, Swarm, Traefik, N..."
902,HeadHunter,Middle/Senior Продуктовый аналитик в направлен...,<p><strong>Продуктовая команда HeadHunter в по...,"[SQL, Tableau, Python, Hadoop, Analytics / DS,..."


In [ ]:
conn = sqlite3.connect(DB_NAME)
query = f'Create table if not Exists {TABLE_NAME} (company_name text, position text, job_description text, key_skills text)'
conn.execute(query)

In [ ]:
df.to_sql(TABLE_NAME,conn,if_exists='append',index=False)